# Azure AI Ained Model Context Protocoli (MCP) toega - Python

See märkmik näitab, kuidas kasutada Azure AI aineid Model Context Protocoli (MCP) tööriistadega Pythonis. See demonstreerib, kuidas luua intelligentne agent, mis suudab kasutada väliseid MCP servereid (näiteks Microsoft Learn) täiustatud funktsioonide jaoks, kasutades võtmeta autentimist.


## Paigalda vajalikud Python'i paketid

Kõigepealt peame paigaldama vajalikud Python'i paketid:
- **azure-ai-projects**: Azure AI Projects SDK põhikomponent
- **azure-ai-agents**: Azure AI Agents SDK agentide loomiseks ja haldamiseks
- **azure-identity**: Võtmeta autentimine, kasutades DefaultAzureCredential'i
- **mcp**: Model Context Protocol'i teostus Python'ile


## Võtmevaba autentimise eelised

See märkmik demonstreerib **võtmevaba autentimist**, mis pakub mitmeid eeliseid:
- ✅ **Pole vaja hallata API võtmeid** - Kasutab Azure'i identiteedipõhist autentimist
- ✅ **Tõhustatud turvalisus** - Saladusi ei salvestata koodi ega konfiguratsioonifailidesse
- ✅ **Automaatne mandaadi uuendamine** - Azure haldab mandaadi elutsükli
- ✅ **Rollipõhine juurdepääsukontroll** - Kasutab Azure RBAC-i täpsete õiguste määramiseks
- ✅ **Mitme keskkonna tugi** - Töötab sujuvalt nii arenduses kui tootmises

`DefaultAzureCredential` valib automaatselt parima saadaoleva mandaadi allika:
1. **Hallatud identiteet** (kui töötab Azure'is)
2. **Azure CLI** mandaadid (kohaliku arenduse ajal)
3. **Visual Studio** mandaadid
4. **Keskkonnamuutujad** (kui need on seadistatud)
5. **Interaktiivne brauseri autentimine** (kui muud võimalused ei tööta)


## Võtmeta autentimise seadistamine

**Võtmeta autentimise eeltingimused:**

### Kohalikuks arenduseks:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure'i keskkondade jaoks:
- Luba **süsteemiga seotud hallatav identiteet** oma Azure'i ressursil
- Määra hallatavale identiteedile sobivad **RBAC rollid**:
  - `Cognitive Services OpenAI User` Azure OpenAI ligipääsuks
  - `AI Developer` Azure AI projektide ligipääsuks

### Keskkonnamuutujad (valikuline):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**API võtmeid ega ühendusstringe pole vaja!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Vajalikud teegid

Impordi vajalikud Python moodulid:
- **os, time**: Standard Python teegid keskkonnamuutujate ja viivituste jaoks
- **AIProjectClient**: Peamine klient Azure AI projektide jaoks
- **DefaultAzureCredential**: Võtmeta autentimine Azure teenuste jaoks
- **MCP-ga seotud klassid**: MCP tööriistade loomise ja haldamise ning kinnituste käsitlemise jaoks


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigureeri MCP serveri seaded

Seadista MCP serveri konfiguratsioon, kasutades keskkonnamuutujaid koos varuväärtustega:
- **MCP_SERVER_URL**: MCP serveri URL (vaikimisi Microsoft Learn API)
- **MCP_SERVER_LABEL**: Silt MCP serveri tuvastamiseks (vaikimisi "mslearn")

See lähenemine võimaldab paindlikku konfiguratsiooni erinevates keskkondades.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI projekti kliendi loomine (võtmeta autentimine)

Azure AI projekti kliendi initsialiseerimine **võtmeta autentimisega**:
- **endpoint**: Azure AI Foundry projekti lõpp-punkti URL
- **credential**: Kasutab `DefaultAzureCredential()` turvaliseks võtmeta autentimiseks
- **API võtmeid pole vaja**: Tuvastab ja kasutab automaatselt parimat saadaolevat mandaati

**Autentimise voog:**
1. Kontrollib hallatud identiteeti (Azure'i keskkondades)
2. Kasutab Azure CLI mandaate (kohalikuks arenduseks)
3. Vajadusel kasutab muid saadaolevaid mandaadi allikaid

See lähenemine välistab vajaduse hallata API võtmeid või ühenduse stringe oma koodis.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Loo MCP tööriista määratlus

Loo MCP tööriist, mis ühendub Microsoft Learn MCP serveriga:
- **server_label**: MCP serveri identifikaator
- **server_url**: MCP serveri URL-i lõpp-punkt
- **allowed_tools**: Valikuline nimekiri, mis piirab, milliseid tööriistu saab kasutada (tühi nimekiri lubab kõiki tööriistu)

See tööriist võimaldab agendil pääseda Microsoft Learn dokumentatsiooni ja ressursside juurde.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Loo agent ja alusta vestlust (võtmeta töövoog)

See põhjalik jaotis näitab täielikku **võtmeta agendi töövoogu**:

1. **Loo AI agent**: Seadista agent GPT-4.1 nano mudeli ja MCP tööriistadega
2. **Loo teema**: Alusta vestlusteemat suhtlemiseks
3. **Saada sõnum**: Küsi agendilt Azure OpenAI ja OpenAI erinevuste kohta
4. **Käsitle tööriistade kinnitusi**: Kinnita automaatselt MCP tööriistade kasutamist, kui vaja
5. **Jälgi täitmist**: Jälgi agendi edenemist ja tegele vajalike toimingutega
6. **Kuva tulemused**: Näita vestluse ja tööriistade kasutamise üksikasju

**Võtmeta funktsioonid:**
- ✅ **Pole kõvakodeeritud salasõnu** - Kogu autentimine toimub Azure identiteedi kaudu
- ✅ **Vaikimisi turvaline** - Kasutab rollipõhist juurdepääsukontrolli
- ✅ **Lihtsustatud juurutamine** - Pole vaja hallata mandaate
- ✅ **Auditisõbralik** - Kogu juurdepääs jälgitakse Azure identiteedi kaudu

Agent kasutab MCP tööriistu, et pääseda Microsoft Learn ressurssidele täieliku turvalisusega ja ilma API võtme haldamiseta.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
